# Example Layouts

This section provides useful layout specifications.

In [1]:
from h5rdmtoolbox import layout
from h5rdmtoolbox.database import hdfdb
import h5rdmtoolbox as h5tbx

Failed to import module h5tbx


## All string dataset must be one-dimensional

In [37]:
lay = layout.Layout()
string_dataset_1D = lay.add(
    hdfdb.FileDB.find,
    flt={'$ndim': {'$eq': 1},
         '$dtype': {'$regex': '^(?!S)*'}},
    recursive=True,
    objfilter='dataset',
    description='String dataset must be 1D',
    n={'$gt': 0}
)

In [38]:
with h5tbx.File() as h5:
    dss = h5.create_string_dataset('a string ds', data=['one', 'two', 'three'])
    dss = h5.create_string_dataset('grp/a string ds', data=['one', 'two', 'three'])
    dsn = h5.create_dataset('a', data=4)
    print(dsn.dtype)
    print(dss.dtype)
    hdf_filename = h5.hdf_filename

int32
|S5


In [39]:
res = lay.validate(hdf_filename)

In [40]:
res.print_summary(exclude_keys=('called', 'kwargs'))


Summary of layout validation
+--------------------------------------+--------+--------------------+---------------------------+---------------+---------------+-----------------------------------------+
| id                                   |   flag | flag description   | description               | target_type   | target_name   | func                                    |
|--------------------------------------+--------+--------------------+---------------------------+---------------+---------------+-----------------------------------------|
| 08d4b8c6-ce67-4a0d-be0d-7c68a353d8e1 |      1 | SUCCESSFUL         | String dataset must be 1D | Group         | /             | h5rdmtoolbox.database.hdfdb.filedb.find |
+--------------------------------------+--------+--------------------+---------------------------+---------------+---------------+-----------------------------------------+
--> Layout is valid


In [41]:
import re

In [43]:
re.search('^(?!S).*', 'M123')

<re.Match object; span=(0, 4), match='M123'>